# Install dependency packages

In [1]:
!pip install transformers -q

# Load tokenizer

In [2]:
import torch

print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.3.1


In [3]:
from transformers import BertTokenizer

PRETRAINED_MODEL_NAME = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 21128


In [4]:
import random
random_tokens = random.sample(list(vocab), 10)
random_ids = [vocab[i] for i in random_tokens]

print("{0:20}{1:15}".format("token", "index"))
print("-" * 25)
for token, index in zip(random_tokens, random_ids):
    print("{0:15}{1:10}".format(token, index))

token               index          
-------------------------
##医                 14335
瓯                    4484
壶                    1901
##赴                 19683
面                    7481
诺                    6437
锂                    7220
##啼                 14639
庭                    2431
傣                     994


# Predict missing word using BERT

In [5]:
sentence = "[CLS] 你知道的，深度学习调参一个玄学 [MASK] 题。"
tokens = tokenizer.tokenize(sentence)
ids = tokenizer.convert_tokens_to_ids(tokens)

print(sentence)
print(tokens)
print(ids)

[CLS] 你知道的，深度学习调参一个玄学 [MASK] 题。
['[CLS]', '你', '知', '道', '的', '，', '深', '度', '学', '习', '调', '参', '一', '个', '玄', '学', '[MASK]', '题', '。']
[101, 872, 4761, 6887, 4638, 8024, 3918, 2428, 2110, 739, 6444, 1346, 671, 702, 4371, 2110, 103, 7579, 511]


In [6]:
import torch
print("PyTorch 版本：", torch.__version__)

from transformers import BertForMaskedLM

# Token tensor对应句子具体内容，segment tensor对应不同句子标识
tokens_tensor = torch.tensor([ids])
segments_tensors = torch.zeros_like(tokens_tensor)
maskedLM_model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)

# 使用模型预测句子缺失内容
maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(tokens_tensor, segments_tensors)
    predictions = outputs[0]
    # (1, seq_len, num_hidden_units)

# 提取出最有可能的前k个缺失内容
masked_index = 16
k = 5
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

print(tokens)
print('-' * 64)
for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
    tokens[masked_index] = t
    print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), tokens))

PyTorch 版本： 1.3.1
['[CLS]', '你', '知', '道', '的', '，', '深', '度', '学', '习', '调', '参', '一', '个', '玄', '学', '[MASK]', '题', '。']
----------------------------------------------------------------
Top 1 (70%)：['[CLS]', '你', '知', '道', '的', '，', '深', '度', '学', '习', '调', '参', '一', '个', '玄', '学', '问', '题', '。']
Top 2 ( 6%)：['[CLS]', '你', '知', '道', '的', '，', '深', '度', '学', '习', '调', '参', '一', '个', '玄', '学', '难', '题', '。']
Top 3 ( 5%)：['[CLS]', '你', '知', '道', '的', '，', '深', '度', '学', '习', '调', '参', '一', '个', '玄', '学', '话', '题', '。']
Top 4 ( 3%)：['[CLS]', '你', '知', '道', '的', '，', '深', '度', '学', '习', '调', '参', '一', '个', '玄', '学', '课', '题', '。']
Top 5 ( 3%)：['[CLS]', '你', '知', '道', '的', '，', '深', '度', '学', '习', '调', '参', '一', '个', '玄', '学', '主', '题', '。']


# Playground

In [7]:
test = True
while(test):
    sentence = input("请输入需要进行填空的句子，其中缺失部分用空格代替：\n")
    masked_index = sentence.find(" ") + 1
    sentence = sentence.replace(" ", " [MASK] ")
    sentence = "[CLS] " + sentence
    print(sentence)
    print('-' * 64)
    
    tokens = tokenizer.tokenize(sentence)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    tokens_tensor = torch.tensor([ids])
    segments_tensors = torch.zeros_like(tokens_tensor)
    
    with torch.no_grad():
        outputs = maskedLM_model(tokens_tensor, segments_tensors)
        predictions = outputs[0]
        
    k = 3
    probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
    
    print(tokens)
    print('-' * 64)
    for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
        tokens[masked_index] = t
        print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), tokens))
    
    operation = input("\n继续？(Y/N)").lower()
    if operation == "yes" or operation == "y":
        print('-' * 64)
    else:
        test = False

del maskedLM_model

请输入需要进行填空的句子，其中缺失部分用空格代替：
卖竹鼠了，三元一 ，十元三只。
[CLS] 卖竹鼠了，三元一 [MASK] ，十元三只。
----------------------------------------------------------------
['[CLS]', '卖', '竹', '鼠', '了', '，', '三', '元', '一', '[MASK]', '，', '十', '元', '三', '只', '。']
----------------------------------------------------------------
Top 1 (93%)：['[CLS]', '卖', '竹', '鼠', '了', '，', '三', '元', '一', '只', '，', '十', '元', '三', '只', '。']
Top 2 ( 1%)：['[CLS]', '卖', '竹', '鼠', '了', '，', '三', '元', '一', '个', '，', '十', '元', '三', '只', '。']
Top 3 ( 0%)：['[CLS]', '卖', '竹', '鼠', '了', '，', '三', '元', '一', '双', '，', '十', '元', '三', '只', '。']

继续？(Y/N)Y
----------------------------------------------------------------
请输入需要进行填空的句子，其中缺失部分用空格代替：
以前 没得选，现在我想做个好人。
[CLS] 以前 [MASK] 没得选，现在我想做个好人。
----------------------------------------------------------------
['[CLS]', '以', '前', '[MASK]', '没', '得', '选', '，', '现', '在', '我', '想', '做', '个', '好', '人', '。']
----------------------------------------------------------------
Top 1 (91%)：['[CLS]', '以', '前', '我', '没', '得